## Basic prompty sample
- Chat with prompty
- Add weather function
- http request using natural language
- code interpreter (powershell)

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.22.0"
#r "nuget: Microsoft.SemanticKernel.Prompty, 1.22.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.22.0"

#r "nuget:AutoGen,0.2.1"
#r "nuget:AutoGen.DotnetInteractive,0.2.1"

Installed Packages AutoGen, 0.2.1 AutoGen.DotnetInteractive, 0.2.1 Microsoft.SemanticKernel, 1.22.0 Microsoft.SemanticKernel.Abstractions, 1.22.0 Microsoft.SemanticKernel.Prompty, 1.22.0-alpha

In [2]:
#r "Interactive.Prompty/Interactive.Prompty/bin/Debug/net8.0/Interactive.Prompty.dll"

In [13]:
Start-Process pwsh -ArgumentList "-c dotnet run --project .\Interactive.Prompty\WeatherService\WeatherForecastingService.csproj"

In [3]:
using Microsoft.DotNet.Interactive;
using Interactive.Prompty;
using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using AutoGen.DotnetInteractive;
using AutoGen.DotnetInteractive.Extension;


await PromptyExtension.LoadExtensionAsync((CompositeKernel)Microsoft.DotNet.Interactive.Kernel.Root);

In [4]:
#!connect prompty --kernel-name prompty

Kernel added: #!prompty

In [5]:
#!connect prompty-orchestrator --kernel-name orchestrator-kernel --prompty-kernel-name prompty --azure-openai-endpoint @input:{"saveAs":"azure-openai-endpoint"} --azure-openai-deployment-name @input:{"saveAs":"azure-openai-deployment-name"} --azure-openai-apikey @password:{"saveAs":"azure-openai-apikey"}

Please enter a value for parameter: --azure-openai-endpoint Please enter a value for parameter: --azure-openai-deployment-name Please enter a value for parameter: --azure-openai-apikey Ok

Kernel added: #!orchestrator-kernel

In [27]:
---
name: ExamplePrompt
description: A prompt that uses context to ground an incoming question
authors:
  - HeHe
model:
  api: chat
sample:
  firstName: Geno
  context: empty
---
 
system:
You are an AI assistant who helps people with_anything. As the assistant,
you answer questions briefly, succinctly, and in a personable manner using markdown and even add some personal flair with appropriate emojis. You try to resolve customer's question in one-step

The following http services are available. You can make request to these services using http request syntax

## weather forecast service
- base address: http://localhost:5049
- swagger
```json
{
    "openapi": "3.0.1",
    "info": {
        "title": "WeatherForecastingService",
        "version": "1.0"
    },
    "paths": {
        "/weatherforecast": {
            "get": {
                "tags": [
                    "WeatherForecastingService"
                ],
                "operationId": "GetWeatherForecast",
                "responses": {
                    "200": {
                        "description": "OK",
                        "content": {
                            "application/json": {
                                "schema": {
                                    "type": "array",
                                    "items": {
                                        "$ref": "#/components/schemas/WeatherForecast"
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    },
    "components": {
        "schemas": {
            "WeatherForecast": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string",
                        "format": "date"
                    },
                    "temperatureC": {
                        "type": "integer",
                        "format": "int32"
                    },
                    "summary": {
                        "type": "string",
                        "nullable": true
                    },
                    "temperatureF": {
                        "type": "integer",
                        "format": "int32",
                        "readOnly": true
                    }
                },
                "additionalProperties": false
            }
        }
    }
}
```
 
# Context
Use the following context to provide a more personalized response to {{firstName}}:
{{context}}
 
user:
{{input}}

In [7]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using System.ComponentModel;
using Microsoft.DotNet.Interactive.Events;
using Microsoft.DotNet.Interactive.Commands;

[KernelFunction]
[Description("Get the weather for a city and state")]
public static Task<string> GetWeather(
    [Description("The city to get the weather for")] string city,
    [Description("The state to get the weather for")] string state)
{
    return Task.FromResult($"The weather in {city}, {state} is 100 degrees");
}

[KernelFunction]
[Description("shut down your computer")]
public async static Task<string> ShutDownPC()
{
    // System.Diagnostics.Process.Start("shutdown","/s /t 0");

    return "PC has been shut down";
}

[KernelFunction]
[Description("Run Powershell")]
public async static Task<string> Powershell([Description("the powershell script to run")] string pwsh)
{
    var events = new List<KernelEvent>();
    var subscriptions = Microsoft.DotNet.Interactive.Kernel.Current.RootKernel.KernelEvents.Subscribe(e => events.Add(e));
    var result = await Microsoft.DotNet.Interactive.Kernel.Root.RunSubmitCodeCommandAsync(pwsh, "pwsh");
    subscriptions.Dispose();
    var displayValues = events
                        .OfType<DisplayEvent>()
                        .SelectMany(x => x.FormattedValues);

    return string.Join("\n", displayValues.Select(x => x.Value)) ?? "Nothing to display";
}

[KernelFunction]
[Description("Run http request")]
public async static Task<string> Http([Description("the .http to run")] string http)
{
    var events = new List<KernelEvent>();
    var subscriptions = Microsoft.DotNet.Interactive.Kernel.Current.RootKernel.KernelEvents.Subscribe(e => events.Add(e));
    var result = await Microsoft.DotNet.Interactive.Kernel.Root.RunSubmitCodeCommandAsync(http, "http");
    subscriptions.Dispose();
    var displayValues = events
                        .OfType<DisplayEvent>()
                        .SelectMany(x => x.FormattedValues);

    return string.Join("\n", displayValues.Select(x => x.Value)) ?? "Nothing to display";
}

In [11]:
Who are you helping with

I'm helping you, Geno! 😊 How can I assist you today?

In [8]:
Hi, what's the weather in newyork

☀️ It's currently 100°F in New York, NY! Stay cool and hydrated! 💧

In [12]:
show me the how many files are in current folder using powershell, list their names

🔍 The current folder has **9 files**. Here are their names:

1. .github
2. Interactive.Prompty
3. .gitignore
4. basic.prompty
5. NuGet.config
6. README.md
7. sample.ipynb
8. z3.ipynb
9. z3.prompty

9
.github
Interactive.Prompty
.gitignore
basic.prompty
NuGet.config
README.md
sample.ipynb
z3.ipynb
z3.prompty


In [28]:
create a weather forecasting report for me

### Weather Forecast Report 🌤️

Here's the upcoming weather forecast:

1. **Date**: 2024-10-18
   - 🌡️ **Temperature**: 28°C (82°F)
   - 📜 **Summary**: Chilly

2. **Date**: 2024-10-19
   - 🌡️ **Temperature**: 43°C (109°F)
   - 📜 **Summary**: Cool

3. **Date**: 2024-10-20
   - 🌡️ **Temperature**: 8°C (46°F)
   - 📜 **Summary**: Sweltering

4. **Date**: 2024-10-21
   - 🌡️ **Temperature**: -19°C (-2°F)
   - 📜 **Summary**: Cool

5. **Date**: 2024-10-22
   - 🌡️ **Temperature**: 29°C (84°F)
   - 📜 **Summary**: Freezing

Stay prepared and have a great day! 🌞

Name,Value
traceparent,00-ac1bbbe1358c188fa513026f92982219-9eb70ec32acdd994-00
Name,Value
Date,"Thu, 17 Oct 2024 07:49:24 GMT"
Server,Kestrel
Transfer-Encoding,chunked
Content-Type,application/json; charset=utf-8
Content-Length,389
date,"""2024-10-18"""
temperatureC,28
summary,"""Chilly"""
